In [1]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=d3f6b2bede9826ff4543ef1063dce71ad092726117e35716e5284e7c1aa60b25
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [3]:
import pandas as pd
import openai
import time
from fpdf import FPDF

from google.colab import userdata

openai.api_key = userdata.get('AIR_Key')

## Load the dataset

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/gianhirakawa/airday4RAGactivity-/refs/heads/master/motorcycle_courier_data.csv')
df.head()

,Service Type,Time,Pick-up Point,Destination (Drop-off),Booking ID,Duration (minutes),Customer Name,Customer Address,Customer Mobile,Recipient Name,Recipient Address,Recipient Mobile,Product Type,Description,Status,Distance (km),Driver Rating,Price (PHP),Driver's Cut,App's Cut
0,Parcel Delivery,12/11/2024 11:47,BGC,Pasig,BKG17455,73.77,Ana De Jesus,789 Burgos St.,9198765432,Mark Tan,123 Main St.,9.217654e+09,Shoes,"Size 9 sneakers, white",On the way to Pick-up Point,13.47,Pending,262.37,209.90,52.47
1,Parcel Delivery,12/11/2024 9:57,San Juan,Caloocan,BKG64506,67.77,Miguel Garcia,10 Rizal Dr.,9234567890,Juan Dela Cruz,10 Rizal Dr.,9.198765e+09,Documents,Confidential file documents,In-transit,10.21,Pending,168.86,135.09,33.77
2,Food Delivery,11/11/2024 4:53,Binondo,Manila,BKG50146,77.52,Juan Dela Cruz,123 Main St.,9127654321,NaN,NaN,NaN,Mang Inasal,Halo-Halo,Completed,11.82,3.2,252.85,202.28,50.57
3,Food Delivery,10/11/2024 15:30,Malate,Quezon City,BKG34424,17.88,Mark Tan,123 Main St.,9198765432,NaN,NaN,NaN,Max's,"Fried Chicken, Pancit Canton, Lumpiang Shanghai",Completed,17.90,3.8,322.93,258.34,64.59
4,Parcel Delivery,10/11/2024 8:20,Malate,Quezon City,BKG73399,78.69,Lisa Macapagal,456 Market Ave.,9234567890,Miguel Garcia,123 Main St.,9.123457e+09,Electronics,Smartphone with charger,Completed,16.22,3.6,277.24,221.79,55.45


## Understanding the Dataset

In [5]:
# Get Column Names and Data Types
column_info = df.dtypes
print(column_info, "\n")


Service Type               object
Time                       object
Pick-up Point              object
Destination (Drop-off)     object
Booking ID                 object
Duration (minutes)        float64
Customer Name              object
Customer Address           object
Customer Mobile             int64
Recipient Name             object
Recipient Address          object
Recipient Mobile          float64
Product Type               object
Description                object
Status                     object
Distance (km)             float64
Driver Rating              object
Price (PHP)               float64
Driver's Cut              float64
App's Cut                 float64
dtype: object 



In [6]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)



Categorical Features: Index(['Service Type', 'Time', 'Pick-up Point', 'Destination (Drop-off)',
       'Booking ID', 'Customer Name', 'Customer Address', 'Recipient Name',
       'Recipient Address', 'Product Type', 'Description', 'Status',
       'Driver Rating'],
      dtype='object')
Numerical Features: Index(['Duration (minutes)', 'Customer Mobile', 'Recipient Mobile',
       'Distance (km)', 'Price (PHP)', 'Driver's Cut', 'App's Cut'],
      dtype='object')


In [7]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display=10):
    uniques = df[col].unique().tolist() #get unique values and turn them into lists
    if len(uniques) > max_display: #if there are more than 10 unique values
        return uniques[:max_display] + ['...']
    return uniques

unique_values = {col: get_unique_values(col) for col in categorical_features}

print("Unique Values for Categorical Features:")
for col, uniques in unique_values.items():
    print(f" - {col}: {uniques})")
print()


Unique Values for Categorical Features:
 - Service Type: ['Parcel Delivery', 'Food Delivery', 'Customer Transportation'])
 - Time: ['12/11/2024 11:47', '12/11/2024 9:57', '11/11/2024 4:53', '10/11/2024 15:30', '10/11/2024 8:20', '10/11/2024 6:47', '10/11/2024 5:40', '09/11/2024 18:07', '09/11/2024 11:52', '09/11/2024 10:15', '...'])
 - Pick-up Point: ['BGC', 'San Juan', 'Binondo', 'Malate', 'Pateros', 'Sta. Ana', 'Intramuros'])
 - Destination (Drop-off): ['Pasig', 'Caloocan', 'Manila', 'Quezon City', 'Paranaque', 'Las Pinas', 'Makati', 'Taguig'])
 - Booking ID: ['BKG17455', 'BKG64506', 'BKG50146', 'BKG34424', 'BKG73399', 'BKG47490', 'BKG24052', 'BKG90067', 'BKG41042', 'BKG59553', '...'])
 - Customer Name: ['Ana De Jesus', 'Miguel Garcia', 'Juan Dela Cruz', 'Mark Tan', 'Lisa Macapagal', 'Liza Lim', 'Jose Rizal', 'Maria Santos', 'Pedro Reyes', 'Carmen Velasco'])
 - Customer Address: ['789 Burgos St.', '10 Rizal Dr.', '123 Main St.', '456 Market Ave.', '89 Kalayaan Rd.'])
 - Recipient Nam

In [8]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n",numerical_stats)



Descriptive Statistics for Numerical Features:
        Duration (minutes)  Customer Mobile  Recipient Mobile  Distance (km)  \
count           20.000000     2.000000e+01      8.000000e+00      20.000000   
mean            60.612500     9.174241e+09      9.185185e+09      10.908000   
std             27.337317     4.634525e+07      3.773285e+07       5.696432   
min             17.880000     9.123457e+09      9.123457e+09       2.420000   
25%             39.442500     9.127654e+09      9.180988e+09       6.550000   
50%             62.645000     9.198765e+09      9.198765e+09      12.120000   
75%             78.732500     9.203488e+09      9.203488e+09      15.455000   
max            109.990000     9.234568e+09      9.217654e+09      18.780000   

       Price (PHP)  Driver's Cut  App's Cut  
count    20.000000     20.000000  20.000000  
mean    221.496000    177.197000  44.299000  
std     107.661246     86.128603  21.532644  
min      77.060000     61.650000  15.410000  
25%     12

## Generating the Template

In [10]:
def generate_template(df,
                      column_info,
                      categorical_features,
                      numerical_features,
                      unique_values,
                      numerical_stats):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types: \n"
    for col, dtype in column_info.items():
        column_summary += f" - {col}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features: \n"
    for col, uniques in unique_values.items():
        unique_values_str += f" - {col}: {uniques}\n"

    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features: \n"
    for col in numerical_features:
        numerical_stats_str += f" - {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"   - {stat_name}: {value}\n"

    # Define the system prompt
    system_prompt = """
        You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
        Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
        Ensure the template is contextually relevant and maintains grammatical correctness.
    """

    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."

    - Output only the template without any explanation or introduction.
    - The template's variables will be dynamically replaced so make sure they're formatted properly.
    """
    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.4,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None

In [11]:
# Generate the template
template = generate_template(df, column_info, categorical_features, numerical_features, unique_values, numerical_stats)
print(template)

"In a {Service Type} service booked on {Time}, {Customer Name} from {Customer Address} requested a pick-up from {Pick-up Point} to {Destination (Drop-off)} with Booking ID {Booking ID}. The delivery, which took approximately {Duration (minutes)} minutes and covered a distance of {Distance (km)} km, involved transporting {Product Type} described as '{Description}'. The total price for this service was {Price (PHP)} PHP, with {Driver's Cut} PHP going to the driver and {App's Cut} PHP retained by the app. The status of the booking is currently '{Status}', and the driver received a rating of {Driver Rating}. For any inquiries, {Customer Name} can be reached at {Customer Mobile}, while the recipient, {Recipient Name}, can be contacted at {Recipient Mobile}."


## Populating the Template with Actual Values

In [12]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key in row_dict:
        if pd.isna(row_dict[key]):
            row_dict[key] = 'N/A'

    # Generate the populated template per row
    paragraph = template.format(**row_dict)

    return paragraph


In [13]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)


In [14]:
df['paragraph'][0]

'"In a Parcel Delivery service booked on 12/11/2024 11:47, Ana De Jesus from 789 Burgos St. requested a pick-up from BGC to Pasig with Booking ID BKG17455. The delivery, which took approximately 73.77 minutes and covered a distance of 13.47 km, involved transporting Shoes described as \'Size 9 sneakers, white\'. The total price for this service was 262.37 PHP, with 209.9 PHP going to the driver and 52.47 PHP retained by the app. The status of the booking is currently \'On the way to Pick-up Point\', and the driver received a rating of Pending. For any inquiries, Ana De Jesus can be reached at 9198765432, while the recipient, Mark Tan, can be contacted at 9217654329.0."'

## Saving the Populated Templates (Natural Language Texts)

In [15]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")

In [16]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [17]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(),
                    txt_filename='nlg.txt')

save_content_to_pdf(df['paragraph'].to_list(),
                   pdf_filename='nlg.pdf')

All content has been saved as 'nlg.txt'.

All content has been saved as 'nlg.pdf'.

